In [1]:
import pandas as pd
import csv, json

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/symptom-disease ds/symptom-disease-1.csv')
df2 = pd.read_csv('/content/drive/MyDrive/symptom-disease ds/symptom-disease-2.csv')
df3 = pd.read_csv('/content/drive/MyDrive/symptom-disease ds/symptom-disease-3.csv')
df4 = pd.read_csv('/content/drive/MyDrive/symptom-disease ds/symptom-disease-4.csv')
df5 = pd.read_csv('/content/drive/MyDrive/symptom-disease ds/symptom-disease-5.csv', on_bad_lines='skip')
df6 = pd.read_excel("/content/drive/MyDrive/symptom-disease ds/symptom-disease-6.xlsx")
df7 = pd.read_csv('/content/drive/MyDrive/symptom-disease ds/symptom-disease-7.csv')
df8 = pd.read_csv('/content/drive/MyDrive/symptom-disease ds/symptom-disease-8.csv')

In [ ]:
#
df1.drop(columns=['cures', 'doctor', 'risk level'], inplace=True)
df1 = df1.rename(columns={'symptoms': 'symptom',})

#
df2 = df2.rename(columns={'label': 'disease', 'text': 'symptom'})
df2 = df2.drop(columns=['Unnamed: 0'], inplace=True)
df3 = df3.rename(columns={'Disease': 'disease', 'Description': 'symptom'})

#
symptom_cols = [f'Symptom_{i}' for i in range(1, 18)]
df4['Symptom'] = df4[symptom_cols].apply(lambda x: ', '.join(x.dropna()), axis=1)
df4 = df4[['Disease', 'Symptom']]
df4 = df4.rename(columns={'Disease': 'disease', 'Symptom': 'symptom'})

#
df5['symptom'] = df5[[f'Symptom_{i}' for i in range(1, 13)]].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)
df5.drop(columns=[f'Symptom_{i}' for i in range(1, 13)], inplace=True)
df5 = df5.rename(columns={'Disease': 'disease'})

#
df6 = df6[['Disease', 'Symptoms']]
df6 = df6.rename(columns={'Disease': 'disease', 'Symptoms': 'symptom'})

#
df7 = df7.rename(columns={'output_text': 'disease', 'input_text': 'symptom'})

#
df8.drop(columns=['disease_id', 'treatment'], inplace=True)
df8 = df8.rename(columns={'common_symptom': 'symptom'})

In [ ]:
# Combine all dataframes
# Save to CSV
final_df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], ignore_index=True)
final_df.to_csv('/content/drive/MyDrive/symptom-disease ds/symptom-disease-final.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/symptom-disease ds/symptom-disease-final.csv')
df.tail()

,disease,symptom
27811,Pediatric Hypersensitivity Vasculitis Syndrome,"Recurrent high fever, fever with rash, joint p..."
27812,Central Neurocytoma,"Lower extremity neurogenic pain, nausea and vo..."
27813,Prepubertal Periodontitis,"In the early stages, patients have minimal pla..."
27814,Pulmonary Hydatid Disease,"Febrile episodes, coughing, expectoration, hem..."
27815,Rectus Sheath Hematoma,Severe lower abdominal pain that gradually spr...


In [ ]:
# this final dataset containg doctors who treate each disease
# was generated using web scraping with python
# the file is in working directory
dffinal = pd.read_csv("/content/drive/MyDrive/symptom-disease ds/symptom-disease-doctor-final.csv")
dffinal.head()

,symptom,disease,doctor
0,"fever,cough,sore throat,runny or stuffy nose,m...",flu,General Physician
1,"cough,mucus production,shortness of breath,che...",bronchitis,General Physician
2,"fever,cough,shortness of breath,chest pain,fat...",pneumonia,General Physician
3,"chest pain,shortness of breath,nausea,vomiting...",heart attack,General Physician
4,"sudden weakness,numbness on one side of the bo...",stroke,General Physician


In [2]:
# Load CSV
df = pd.read_csv("/content/drive/MyDrive/model-data/symptom-disease-doctor-final.csv")
output_path = "/content/drive/MyDrive/model-data/symptom-disease-doctor-final.jsonl"

with open(output_path, "w") as f:
    for _, row in df.iterrows():
        user_msg = f"I have {row['symptom']}."
        model_msg = f"Based on your symptoms, you likely have {row['disease']}. I recommend you see a {row['doctor']}."
        example = {
            "messages": [
                {"role": "user", "content": user_msg},
                {"role": "model", "content": model_msg}
            ]
        }
        f.write(json.dumps(example) + "\n")

print(f"JSONL file created successfully at {output_path}")


JSONL file created successfully at /content/drive/MyDrive/model-data/symptom-disease-doctor-final.jsonl
